# Data Cleaning - Broadband V. Mobile, Coverage, and Mobile Phone Cost

In [23]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import patheffects
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas
from scipy.stats import linregress

data_to_load = Path("Resources/mobile_data_index_breakdown.csv")

internet_adoption_data = pd.read_csv(data_to_load)

# Replace 'inf' with NaN and drop rows with NaN 
internet_data_clean = internet_adoption_data.replace([np.inf, -np.inf], np.nan).dropna()

#minor data cleaning - renaming columns
new_columns = {'1.1.1) Internet users': 'Internet Users', 
               '1.1.2) Fixed-line broadband subscribers': 'Broadband Subscribers',
               '1.1.3) Mobile subscribers': 'Mobile Subscribers',
               '1.3.1) Network coverage (min. 2G)': '2G Coverage',
               '1.3.2) Network coverage (min. 3G)': '3G Coverage',
               '1.3.3) Network coverage (min. 4G)': '4G Coverage',
               '2.1.2) Mobile phone cost (prepaid tariff)': 'Mobile Phone Cost (Prepaid)',
               '2.1.3) Mobile phone cost (postpaid tariff)': 'Mobile Phone Cose (Postpaid)',
               '2.1.4) Fixed-line monthly broadband cost': 'Broadband Cost',
               'BG2) Population': 'Population'
              }
internet_data_clean.rename(columns=new_columns, inplace=True)

internet_data_clean

,Country,Edition,Internet Users,Broadband Subscribers,Mobile Subscribers,2G Coverage,3G Coverage,4G Coverage,Mobile Phone Cost (Prepaid),Mobile Phone Cose (Postpaid),Broadband Cost,Population
0,Algeria,2017,31.9,5.71,108.8,99.19,46.00,0.00,5.501,4.914,3.82,40.6
1,Algeria,2018,47.3,7.05,116.0,98.00,83.36,3.62,5.501,2.007,4.01,41.3
2,Algeria,2019,54.2,7.76,110.8,98.00,90.00,30.49,2.038,2.207,4.41,42.1
3,Algeria,2020,74.4,7.26,111.7,98.04,97.45,52.84,1.981,2.207,4.41,42.9
4,Algeria,2021,74.4,8.32,109.4,98.04,97.72,53.63,2.379,2.207,4.41,43.7
...,...,...,...,...,...,...,...,...,...,...,...,...
595,Zimbabwe,2017,21.6,1.19,92.3,88.70,55.10,11.00,44.680,75.000,14.06,14.0
596,Zimbabwe,2018,22.1,1.22,91.8,88.90,55.25,22.64,32.970,32.558,13.95,14.2
597,Zimbabwe,2019,24.0,1.32,99.0,90.09,78.20,34.72,19.780,30.882,13.24,14.4
598,Zimbabwe,2020,30.3,1.41,89.4,93.00,84.00,35.00,10.056,30.882,13.24,14.6


## Data Cleaning

In [90]:
country = internet_data_clean['Country'].unique()

In [91]:
#Calculate the total average of population for each country
population = internet_data_clean.groupby('Country')['Population'].mean()

### Broadband and Mobile Subscribers

In [92]:
# Calculate the total mean for each country for broadband
clean_broadband = internet_data_clean.groupby('Country')['Broadband Subscribers'].mean()

In [93]:
#Normalize the broadband subscribers per population
broadband_population = clean_broadband * population / 100

In [94]:
# Calculate the total mean for each country for mobile
clean_mobile = internet_data_clean.groupby('Country')['Mobile Subscribers'].mean()

In [95]:
#Normalize mobile subscribers per population
mobile_population = clean_mobile * population / 100

### 2g, 3g, 4g, Service

In [96]:
# Calculate the total average for each country for 2G
twog_data = internet_data_clean.groupby('Country')['2G Coverage'].mean()

In [97]:
# Calculate the total average for each country for 3G
threeg_data = internet_data_clean.groupby('Country')['3G Coverage'].mean()

In [98]:
# Calculate the total average for each country for 4G
fourg_data = internet_data_clean.groupby('Country')['4G Coverage'].mean()

### Broadband v. Mobile Cost

In [99]:
#Calculate broadband cost
broadband_cost = internet_data_clean.groupby('Country')['Broadband Cost'].mean()

In [100]:
#Calculate mobile phone cost - averaging prepaid and postpaid
prepaid = internet_data_clean.groupby('Country')['Mobile Phone Cost (Prepaid)'].mean()
postpaid = internet_data_clean.groupby('Country')['Mobile Phone Cose (Postpaid)'].mean()

mobile_cost = prepaid * postpaid / 2

## Data Summary

In [101]:
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
internet_summary = pd.DataFrame(
    {
        "Country" : country,
        "Broadband Subscribers" : broadband_population,
        "Mobile Subscribers" : mobile_population,
        "2G Coverage" : twog_data,
        "3G Coverage" : threeg_data,
        "4G Coverage" : fourg_data,
        "Broadband Cost (% of GNI)" : broadband_cost,
        "Mobile Cost (% of GNI)" : mobile_cost
    }
)

# Minor data wrangling
internet_summary = internet_summary.reset_index(drop=True)

# Minor data cleaning
internet_summary["Broadband Subscribers"] = internet_summary["Broadband Subscribers"].map("{:,.2f}".format)
internet_summary["Mobile Subscribers"] = internet_summary["Mobile Subscribers"].map("{:,.2f}".format)
internet_summary["2G Coverage"] = internet_summary["2G Coverage"].map("{:,.2f}".format)
internet_summary["3G Coverage"] = internet_summary["3G Coverage"].map("{:,.2f}".format)
internet_summary["4G Coverage"] = internet_summary["4G Coverage"].map("{:,.2f}".format)
internet_summary["Broadband Cost (% of GNI)"] = internet_summary["Broadband Cost (% of GNI)"].map("{:,.2f}".format)
internet_summary["Mobile Cost (% of GNI)"] = internet_summary["Mobile Cost (% of GNI)"].map("{:,.2f}".format)

#export to CSV
#internet_summary.to_csv("Resources/internet_summary.csv")

# Display the DataFrame
internet_summary

,Country,Broadband Subscribers,Mobile Subscribers,2G Coverage,3G Coverage,4G Coverage,Broadband Cost (% of GNI),Mobile Cost (% of GNI)
0,Algeria,3.04,46.90,98.25,82.91,28.12,4.21,4.71
1,Angola,0.12,14.13,91.80,72.93,9.80,12.63,30.57
2,Argentina,7.91,61.12,98.00,94.00,83.39,3.61,0.47
3,Australia,7.95,27.25,99.32,99.32,97.92,1.15,0.07
4,Austria,2.45,11.18,99.00,98.00,98.00,0.74,0.01
...,...,...,...,...,...,...,...,...
115,Uzbekistan,3.35,25.53,98.52,63.91,32.32,2.41,15.57
116,Venezuela,2.68,23.72,94.20,86.86,64.76,3.39,9.40
117,Vietnam,11.24,128.71,97.38,88.94,58.18,0.53,1.60
118,Zambia,0.05,14.30,90.56,61.46,28.61,23.19,41.08


## Broadband Top 10 Countries

In [88]:
#Find the top 10 Broadband Countries
broadband_top_ten = broadband_population.nlargest(10)

In [89]:
#Find the last 10 Broadband Countries
broadband_last_ten = broadband_population.nsmallest(10)

In [50]:
#Add Data to two data frames
broadband_top = pd.DataFrame({"Broadband Top 10" : broadband_top_ten})
broadband_bottom = pd.DataFrame({"Broadband Bottom 10" : broadband_last_ten})

# Minor data cleaning - renaming
broadband_top = broadband_top.rename_axis('Top Country')
broadband_bottom = broadband_bottom.rename_axis('Bottom Country')

# Minor data cleaning - adding columns and values
broadband_top["Broadband Top 10"] = broadband_top["Broadband Top 10"].map("{:,.2f}".format)
broadband_top['Place'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
broadband_top['Number'] = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
broadband_bottom["Broadband Bottom 10"] = broadband_bottom["Broadband Bottom 10"].map("{:,.2f}".format)
broadband_bottom['Place'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
broadband_bottom['Number'] = [-10, -9, -8, -7, -6, -5, -4, -3, -2, -1]

# Display the DataFrame
display(broadband_top, broadband_bottom)

#Export both singles to one CSV
broadband_top.to_csv("Resources/Broadband_10.csv")
broadband_bottom.to_csv("Resources/Broadband_10.csv", mode='a', header=True)

,Broadband Top 10,Place,Number
Top Country,,,
China,360.17,1,10
United States,108.89,2,9
Japan,40.56,3,8
Germany,33.04,4,7
Russia,30.75,5,6
Brazil,29.15,6,5
France,28.44,7,4
United Kingdom,26.02,8,3
South Korea,21.03,9,2


,Broadband Bottom 10,Place,Number
Bottom Country,,,
Guinea,0.00,1,-10
Congo (DRC),0.00,2,-9
Burundi,0.00,3,-8
Liberia,0.01,4,-7
Malawi,0.01,5,-6
Burkina Faso,0.01,6,-5
Niger,0.01,7,-4
Rwanda,0.02,8,-3
Papua New Guinea,0.02,9,-2


In [102]:
#Import the complete CSV
data_to_load = Path("Resources/Broadband_10.csv")

broadband_10_data = pd.read_csv(data_to_load)
broadband_10_data


,Top Country,Broadband Top 10,Place,Number,Bottom Country,Broadband Bottom 10,Place.1,Number.1
0,China,360.17,1,10,Guinea,0.00,1,-10
1,United States,108.89,2,9,Congo (DRC),0.00,2,-9
2,Japan,40.56,3,8,Burundi,0.00,3,-8
3,Germany,33.04,4,7,Liberia,0.01,4,-7
4,Russia,30.75,5,6,Malawi,0.01,5,-6
5,Brazil,29.15,6,5,Burkina Faso,0.01,6,-5
6,France,28.44,7,4,Niger,0.01,7,-4
7,United Kingdom,26.02,8,3,Rwanda,0.02,8,-3
8,South Korea,21.03,9,2,Papua New Guinea,0.02,9,-2
9,India,18.31,10,1,Gabon,0.02,10,-1


## Mobile Top 10 Countries

In [85]:
#Find the top 10 Mobile Countries
mobile_top_ten = mobile_population.nlargest(10)

In [103]:
#Find the last 10 Mobile Countries
mobile_last_ten = mobile_population.nsmallest(10)

In [53]:
#Add Data to two data frames
mobile_top = pd.DataFrame({"Mobile Top 10" : mobile_top_ten})
mobile_bottom = pd.DataFrame({"Mobile Bottom 10" : mobile_last_ten})

# Minor data cleaning - renaming
mobile_top = mobile_top.rename_axis('Top Country')
mobile_bottom = mobile_bottom.rename_axis('Bottom Country')

# Minor data cleaning - adding columns and values
mobile_top["Mobile Top 10"] = mobile_top["Mobile Top 10"].map("{:,.2f}".format)
mobile_top['Place'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
mobile_top['Number'] = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
mobile_bottom["Mobile Bottom 10"] = mobile_bottom["Mobile Bottom 10"].map("{:,.2f}".format)
mobile_bottom['Place'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
mobile_bottom['Number'] = [-10, -9, -8, -7, -6, -5, -4, -3, -2, -1]

# Display the DataFrame
display(mobile_top, mobile_bottom)

#Export both singles to one CSV
mobile_top.to_csv("Resources/Mobile_10.csv")
mobile_bottom.to_csv("Resources/Mobile_10.csv", mode='a', header=True)


,Mobile Top 10,Place,Number
Top Country,,,
China,"1,464.27",1,10
India,"1,136.05",2,9
United States,410.92,3,8
Indonesia,361.96,4,7
Russia,235.72,5,6
Brazil,227.95,6,5
Japan,172.96,7,4
Nigeria,165.52,8,3
Bangladesh,150.52,9,2


,Mobile Bottom 10,Place,Number
Bottom Country,,,
Estonia,1.89,1,-10
Trinidad & Tobago,2.12,2,-9
Latvia,2.35,3,-8
Bahrain,2.37,4,-7
Namibia,2.73,5,-6
Jamaica,2.86,6,-5
Gabon,2.96,7,-4
Liberia,3.08,8,-3
Botswana,3.48,9,-2


In [55]:
#Import the complete CSV
data_to_load = Path("Resources/Broadband_10.csv")

broadband_10_data = pd.read_csv(data_to_load)
broadband_10_data

,Top Country,Broadband Top 10,Place,Number,Bottom Country,Broadband Bottom 10,Place.1,Number.1
0,China,360.17,1,10,Guinea,0.00,1,-10
1,United States,108.89,2,9,Congo (DRC),0.00,2,-9
2,Japan,40.56,3,8,Burundi,0.00,3,-8
3,Germany,33.04,4,7,Liberia,0.01,4,-7
4,Russia,30.75,5,6,Malawi,0.01,5,-6
5,Brazil,29.15,6,5,Burkina Faso,0.01,6,-5
6,France,28.44,7,4,Niger,0.01,7,-4
7,United Kingdom,26.02,8,3,Rwanda,0.02,8,-3
8,South Korea,21.03,9,2,Papua New Guinea,0.02,9,-2
9,India,18.31,10,1,Gabon,0.02,10,-1


## Broadband Top 10 v. Mobile Top 10

In [56]:
# Display the DataFrame
display(broadband_top, mobile_top)

,Broadband Top 10,Place,Number
Top Country,,,
China,360.17,1,10
United States,108.89,2,9
Japan,40.56,3,8
Germany,33.04,4,7
Russia,30.75,5,6
Brazil,29.15,6,5
France,28.44,7,4
United Kingdom,26.02,8,3
South Korea,21.03,9,2


,Mobile Top 10,Place,Number
Top Country,,,
China,"1,464.27",1,10
India,"1,136.05",2,9
United States,410.92,3,8
Indonesia,361.96,4,7
Russia,235.72,5,6
Brazil,227.95,6,5
Japan,172.96,7,4
Nigeria,165.52,8,3
Bangladesh,150.52,9,2


In [ ]:
#Future Chart

## Top 10 Countries 2g, 3g, and 4g Coverage

In [83]:
#Set the Id's for the search
g_ids = ['China', 'India', 'United States', 'Indonesia', 'Russia', 'Brazil', 'Japan', 'Nigeria', 'Bangladesh', 'Pakistan']

#Search the DataFrame and return the results from the appended list
mobile_coverage = internet_summary[internet_summary['Country'].isin(g_ids)][['Country','2G Coverage', '3G Coverage', '4G Coverage']]

#Set the new order of the DataFrame
order = ['China', 'India', 'United States', 'Indonesia', 'Russia', 'Brazil', 'Japan', 'Nigeria', 'Bangladesh', 'Pakistan']
coverage_reordered = mobile_coverage.set_index('Country').reindex(order).reset_index()

#Show
coverage_reordered

,Country,2G Coverage,3G Coverage,4G Coverage
0,China,99.66,98.06,95.86
1,India,96.81,86.76,71.48
2,United States,99.90,99.90,99.76
3,Indonesia,94.36,84.81,62.64
4,Russia,91.98,78.14,65.74
5,Brazil,94.40,94.17,80.46
6,Japan,99.90,99.90,99.00
7,Nigeria,92.33,59.54,17.21
8,Bangladesh,99.51,89.16,71.60
9,Pakistan,87.70,70.92,49.47


In [ ]:
#Future Chart

### Top 10 Countries Cost of Broadband v. Cost of Mobile

In [84]:
#Set the Id's for the search
g_ids = ['China', 'India', 'United States', 'Indonesia', 'Russia', 'Brazil', 'Japan', 'Nigeria', 'Bangladesh', 'Pakistan']

#Search the DataFrame and return the results from the appended list
mobile_coverage = internet_summary[internet_summary['Country'].isin(g_ids)][['Country', 'Broadband Cost (% of GNI)', 'Mobile Cost (% of GNI)']]

#Set the new order of the DataFrame
order = ['China', 'India', 'United States', 'Indonesia', 'Russia', 'Brazil', 'Japan', 'Nigeria', 'Bangladesh', 'Pakistan']
coverage_reordered = mobile_coverage.set_index('Country').reindex(order).reset_index()

#Show
coverage_reordered

,Country,Broadband Cost (% of GNI),Mobile Cost (% of GNI)
0,China,2.34,0.85
1,India,4.60,2.18
2,United States,0.84,0.15
3,Indonesia,8.94,0.85
4,Russia,0.55,0.24
5,Brazil,2.09,2.18
6,Japan,0.83,1.74
7,Nigeria,14.61,2.43
8,Bangladesh,3.67,3.22
9,Pakistan,5.94,3.32


In [ ]:
#Future Chart Here